## Portfolio Optimisation

Firstly, let's talk about portfolio optimisation then we can move onto the Modern Portfolio Theory. Portfolio optimisation can just be defined as "how to maxmise an objective function that relates to your portfolio while ensuring that the weights sum to 1".

Now that objective function can be anything that can be defined by the user. It can be the risk adjusted returns of your whole portfolio, it can be just maximising returns or it can be to minimise the overall variance of your portfolio.

In mathematical terms, the problem can be stated as:
$
\max_{weights} f(X, weights)
$
subject to the contraints:
$
\sum_{i} weights_{i} = 1
$

where $f(X, weights)$ is your objective function and $X$ is the returns dataframe of your tickers.

## Modern Portfolio Theory

Modern Portfolio Theory (MPT) is an investment theory developed by Harry Markowitz and published under the title "Portfolio Selection" in the Journal of Finance in 1952.

There are a few underlying concepts that can help anyone to understand MPT. If you are familiar with finance, you might know what the acronym "TANSTAAFL" stands for. It is a famous acronym for "There Ain't No Such Thing As A Free Lunch". This concept is also closely related to 'risk-return trade-off'.

Higher risk is associated with greater probability of higher return and lower risk with a greater probability of smaller return. MPT assumes that investors are risk-averse, meaning that given two portfolios that offer the same expected return, investors will prefer the less risky one. Thus, an investor will take on increased risk only if compensated by higher expected returns.

Another factor comes in to play in MPT is "diversification". Modern portfolio theory says that it is not enough to look at the expected risk and return of one particular stock. By investing in more than one stock, an investor can reap the benefits of diversification – chief among them, a reduction in the riskiness of the portfolio.

What you need to understand is "risk of a portfolio is not equal to average/weighted-average of individual stocks in the portfolio". In terms of return, yes it is the average/weighted average of individual stock's returns, but that's not the case for risk. The risk is about how volatile the asset is, if you have more than one stock in your portfolio, then you have to take count of how these stocks movement correlates with each other. The beauty of diversification is that you can even get lower risk than a stock with the lowest risk in your portfolio, by optimising the allocation.

In [3]:
import yfinance as yfin
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.optimize as sco
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

plt.style.use("fivethirtyeight")
%config InlineBackend.figure_format = 'retina'

In [5]:
# Define any random tickers that we have in our portfolio
tickers = ['AAPL','AMZN','GOOGL','META']

# Download the tickers using Yahoo Finance
dataframe = yfin.download(tickers, period='1y')['Adj Close']

# First fill forward and then drop the NA values
dataframe = dataframe.ffill().dropna()

[*********************100%***********************]  4 of 4 completed


In [6]:
dataframe.head()

AAPL        AMZN       GOOGL        META
Date                                                                     
2021-10-12 00:00:00-04:00  140.721466  162.366501  136.449005  323.769989
2021-10-13 00:00:00-04:00  140.124802  164.214005  137.582001  324.540009
2021-10-14 00:00:00-04:00  142.958908  164.992996  141.151001  328.529999
2021-10-15 00:00:00-04:00  144.032883  170.451004  141.367996  324.760010
2021-10-18 00:00:00-04:00  145.733383  172.337006  142.778000  335.339996

In [7]:
# Taking the daily returns of our dataframe
returns = dataframe.pct_change().dropna()

returns.head()

AAPL      AMZN     GOOGL      META
Date                                                             
2021-10-13 00:00:00-04:00 -0.004240  0.011379  0.008303  0.002378
2021-10-14 00:00:00-04:00  0.020226  0.004744  0.025941  0.012294
2021-10-15 00:00:00-04:00  0.007512  0.033080  0.001537 -0.011475
2021-10-18 00:00:00-04:00  0.011806  0.011065  0.009974  0.032578
2021-10-19 00:00:00-04:00  0.015080 -0.000751  0.003215  0.013867

Let us look at the price series of our dataframe and how the Adj Close price has been this past year

## Why Variance is not linear in summation?


$
Var(X + Y) \neq Var(X) + Var(Y)
$

Variance is not linear under the summation sign, there is the additional term of covariance between the two random variables that also exists.

$
Var(X + Y) = Var(X) + Var(Y) + 2Cov(X, Y)
$

and because of this fact, we see that higher risk may not always mean a higher return, sometimes, we can get a higher return from our portfolio while keeping the volatility of the portfolio constant.


## Efficient Frontier
From the plot of the randomly simulated portfolio, we can see it forms a shape of an arch line on the top of clustered blue dots. This line is called efficient frontier. Why is it efficient? Because points along the line will give you the lowest risk for a given target return. All the other dots right to the line will give you higher risk with same returns. If the expected returns are the same, why would you take an extra risk when there's an option with lower risk?

The way we found the two kinds of optimal portfolio above was by simulating many possible random choices and pick the best ones (either minimum risk or maximum risk-adjusted return). We can also implement this by using Scipy's optimize function.

If you are an advanced Excel user, you might be familiar with 'solver' function in excel. Scipy's optimize function is doing the similar task when given what to optimize, and what are constraints and bounds.

Below functions are to get the maximum Sharpe ratio portfolio. In Scipy's optimize function, there's no 'maximize', so as an objective function you need to pass something that should be minimized. That is why the first "neg_sharpe_ratio" is computing the negative Sharpe ratio. Now we can use this as our objective function to minimize. In "max_sharpe_ratio" function, you first define arguments (this should not include the variables you would like to change for optimisation, in this case, "weights"). At first, the construction of constraints was a bit difficult for me to understand, due to the way it is stated.

constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

The above constraint is saying that sum of x should be equal to 1. You can think of the 'fun' part construction as '1' on the right side of equal sign has been moved to the left side of the equal sign.

'np.sum(x) == 1' has become 'np.sum(x)-1'

And what does this mean? It simply means that the sum of all the weights should be equal to 1. You cannot allocate more than 100% of your budget in total.

"bounds" is giving another limit to assign random weights, by saying any weight should be inclusively between 0 and 1. You cannot give minus budget allocation to a stock or more than 100% allocation to a stock.

## How to build your own Mean Variance portfolio

In this tutorial, we will demonstate how to build your own portfolio that maximises your sharpe ratio or one that minimizes the total variance of your portfolio

In [16]:
from financial_ratios import *
from stats import *
from annualize import *

In [17]:
def neg_sharpe_ratio(weights, dataframe, risk_free_rate = 0):

    # Multiply prices by the weights assigned to them and then sum across each date to get portfolio value
    portfolio = (weights * dataframe).sum(axis=1)

    # Calculate the sharpe ratio of our portfolio and return the negative of that
    return -sharpe_ratio(portfolio, risk_free_rate)

In [18]:
def max_sharpe_ratio(dataframe, risk_free_rate = 0):

    # How many assets do we have in our portfolio
    num_assets = len(dataframe.columns)

    # What are the additional arguements that our objective function needs
    args = (dataframe)
    # What are the constraints on our optimisation
    # Here it is that the sum of the weights must equal to one
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    # What are the individual bouds for each weight
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [19]:
max_sharpe_ratio(dataframe)

     fun: 0.03845560941529632
     jac: array([-0.        ,  1.06383583,  0.83587475,  4.22404591])
 message: 'Optimization terminated successfully'
    nfev: 20
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([1., 0., 0., 0.])

In [20]:
def portfolio_volatility(weights,dataframe):

    # Calculate our portfolio returns for each date
    portfolio = (dataframe.pct_change().dropna() * weights).sum(axis=1)

    return annualised_volatility(portfolio)

def min_variance(dataframe):

    num_assets = len(dataframe.columns)

    args = (dataframe)

    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

In [21]:
min_variance(dataframe)

     fun: 0.3118544573169744
     jac: array([0.31185802, 0.34421779, 0.31184792, 0.37068996])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([6.47644875e-01, 3.94649591e-17, 3.52355125e-01, 0.00000000e+00])

In [22]:
min_var_weights = min_variance(dataframe)['x']
max_sharpe_weights = max_sharpe_ratio(dataframe)['x']


In [23]:
def backtest(dataframe, weights):

    portfolio = (dataframe * weights).sum(axis=1)
    portfolio = 1000 * (portfolio / portfolio.iloc[0])

    return portfolio

In [24]:
min_vol_backtest = backtest(dataframe, min_var_weights)
max_sharpe_backtest = backtest(dataframe, max_sharpe_weights)

In [25]:
fig, ax = plt.subplots(figsize=(16,8))
min_vol_backtest.plot(ax=ax)
max_sharpe_backtest.plot(ax=ax)
plt.legend(['Min Vol', 'Max Sharpe'])
